In [ ]:
import os
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import time
import copy

from torch.optim import lr_scheduler
from torchvision import datasets
from efficientnet_pytorch import EfficientNet
from torch.utils.tensorboard import SummaryWriter

In [ ]:
batch_size = 16
epochs = 101
input_path = '/home/pirl/필사본_NotZero/'
writer = SummaryWriter('./runs/experiment1/')

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(input_path, x),
                                          data_transforms[x])
                  for x in ['train', 'validation']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'validation']}


In [ ]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes = 1108)
model.to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
best_model_weights = copy.deepcopy(model.state_dict())
best_acc = 0.0

In [ ]:
for epoch in range(epochs):
    since = time.time()
    print('Epoch {}/{}'.format(epoch, epochs - 1))
    print('-' * 10)

    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

        torch.save(model.state_dict(), './weights/best_weights_b5_class_{}.pth'.format(epoch))
        if phase == 'validation' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_weights = copy.deepcopy(model.state_dict())
    time_elapsed = time.time() - since
    print("time : ", time_elapsed)
    print()

Epoch 0/100
----------
train Loss: 0.8143 Acc: 0.8524
validation Loss: 0.6057 Acc: 0.9207
time :  2100.25293302536

Epoch 1/100
----------
train Loss: 0.4010 Acc: 0.9290
validation Loss: 0.5716 Acc: 0.9266
time :  2080.6075434684753

Epoch 2/100
----------
train Loss: 0.3435 Acc: 0.9397
validation Loss: 0.5071 Acc: 0.9329
time :  2080.58993434906

Epoch 3/100
----------
train Loss: 0.3108 Acc: 0.9460
validation Loss: 0.4950 Acc: 0.9283
time :  2078.3389921188354

Epoch 4/100
----------
train Loss: 0.2923 Acc: 0.9490
validation Loss: 0.4656 Acc: 0.9368
time :  2080.472844362259

Epoch 5/100
----------
train Loss: 0.2768 Acc: 0.9518
validation Loss: 0.4497 Acc: 0.9329
time :  2078.343804836273

Epoch 6/100
----------
train Loss: 0.2605 Acc: 0.9535
validation Loss: 0.4499 Acc: 0.9353
time :  2079.5339920520782

Epoch 7/100
----------
train Loss: 0.1940 Acc: 0.9641
validation Loss: 0.3926 Acc: 0.9443
time :  2081.52205991745

Epoch 8/100
----------
train Loss: 0.1763 Acc: 0.9660
validation

validation Loss: 0.3541 Acc: 0.9443
time :  2103.0052132606506

Epoch 70/100
----------
train Loss: 0.1241 Acc: 0.9727
validation Loss: 0.3544 Acc: 0.9444
time :  2159.5259346961975

Epoch 71/100
----------
train Loss: 0.1233 Acc: 0.9731
validation Loss: 0.3540 Acc: 0.9443
time :  2160.4458060264587

Epoch 72/100
----------
train Loss: 0.1226 Acc: 0.9731
validation Loss: 0.3537 Acc: 0.9443
time :  2086.010239124298

Epoch 73/100
----------
train Loss: 0.1237 Acc: 0.9730
validation Loss: 0.3538 Acc: 0.9444
time :  2075.3960258960724

Epoch 74/100
----------
train Loss: 0.1236 Acc: 0.9731
validation Loss: 0.3542 Acc: 0.9443
time :  2094.6518318653107

Epoch 75/100
----------
train Loss: 0.1238 Acc: 0.9730
validation Loss: 0.3540 Acc: 0.9443
time :  2091.6363253593445

Epoch 76/100
----------
train Loss: 0.1234 Acc: 0.9730
validation Loss: 0.3541 Acc: 0.9443
time :  2067.9217483997345

Epoch 77/100
----------
train Loss: 0.1241 Acc: 0.9732
validation Loss: 0.3539 Acc: 0.9445
time :  2066.

In [ ]:
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

torch.save(best_model_weights, './weights/best_weights_b5_class_15.pth')